# Sexism Identification in Social networks (EXIST 2021).  

## Entrega Final: Trayectoria en el proyecto y exposición de resultados finales: 

  
Autores:
Andrea García Pastor  
Elizaveta Gilyarovskaya 

A continuación expondremos los pasos seguidos y los métodos utilizados a lo largo de las últimas sesiones para llegar hasta el modelo final que hemos utilizado para predecir el conjunto test que vamos a enviar a la competición.


In [ ]:
# Descargamos librerías
import pandas as pd
import numpy as np
import re
import string
import math

# Preprocesamiento
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from nltk.stem import PorterStemmer
#from nltk.stem import WordNetLemmatizer

# Machine Learning 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from gensim.models import Word2Vec
#from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.naive_bayes import MultinomialNB 
#from sklearn.linear_model import LogisticRegression 
#from sklearn.svm import SVC 
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#from mlxtend.regressor import StackingCVRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Para Andrea
%cd "/content/drive/My Drive/lnr"

/content/drive/My Drive/lnr


In [ ]:
# Para Lisa
%cd "/content/drive/My Drive/lnr bert"


[Errno 2] No such file or directory: '/content/drive/My Drive/lnr bert'
/content


### 1. Lo primero es leer el training dataset.   
Para este proyecto hemos decidido entrenar un clasificador únicamente para los tweets ingleses. Por tanto filtramos por la variable 'language' aquellas filas que eran de tweets españoles. Debido al tiempo reducido, esta decisión viene motivada por el hecho de querer aprender a entrenar un clasificador para el Natural Language Processing independientemente del idioma, nos interesaba dedicar más tiempo a probar diferentes estrategias y modelos de natural language processing que dedicar tiempo a hacer el mismo modelo para dos idiomas distintos.

In [ ]:
#Leemos el training dataset:
def read_tsv(url, language): 
    df = pd.read_csv(url, sep='\t')
    df = df[df['language'] == language]
    texts = list(df['text']) 
    ids = list(df['id'])
    t1 = list(df['task1']) 
    t2 = list(df['task2']) 
    return texts, ids,  t1, t2

### 2. Preproceso

El preprocesamiento de los tweets va a consistir en quitar los signos de puntuación, los dígitos, quitar los enlaces a las páginas web, las menciones y los hashtags. Las funciones implementadas para eso son las que se muestran a continuación:

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))

def regex(tweet):  # Quitamos URL, @, emoticonos, dígitos y #
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+www\S+|https?://[\S]+", '', tweet, flags=re.MULTILINE)  
    tweet = re.sub(r'\@\w+|\#','', tweet) 
    tweet = re.sub(r"[\U00010000-\U0010ffff]|xD|XD|:(|:)|;(|;)|:D",'', tweet) 
    tweet = re.sub(r'[0-9]+','', tweet) 
    tweet = tweet.translate(str.maketrans('','', string.punctuation))
    #tweet = re.sub(r"([\.]+|[!`¡#$%&\'´’¿*+,-./(...):;<=*>?@\^_`~])", '', tweet) #no va bien del todo
    return tweet


def preprocessing(tweet): 
    clean_tweet = regex(tweet)
    # Quitamos stopwords 
    tokens= word_tokenize(clean_tweet) 
    filtered = [w for w in tokens if not w in stop_words]
    return " ".join(filtered)

Importamos el training set y aplicamos el preproceso, miramos algunos de los tweets:

In [ ]:
tweets, ids, t1, t2 = read_tsv('EXIST2021_training.tsv','en') 

Veamos el resultado del preproceso:

In [ ]:
print('Tweet real:', tweets[23])
print('Tweet preprocesado:', preprocessing(tweets[23]), '\n')

print('Tweet real:', tweets[5])
print('Tweet preprocesado:', preprocessing(tweets[5]), '\n') 

print('Tweet real:', tweets[3])
print('Tweet preprocesado:', preprocessing(tweets[3]), '\n') 


Tweet real: @Banyosssss My philosophy: He has a funny mustache what else does a man need
Tweet preprocesado: philosophy funny mustache else man need 

Tweet real: @Smithcouple971 Hello....m raj....m with good size and excellent stamina ....A passionate pussy licker...Love to lick every holes were womens desire to b licked...DoggyMissionaryWomen on topMy best pose to spank over bed
Tweet preprocesado: hellom rajm good size excellent stamina passionate pussy lickerlove lick every holes womens desire b lickeddoggymissionarywomen topmy best pose spank bed 

Tweet real: @AurelieGuiboud Incredible!  Beautiful!But I laughed so much when I read about you drifting in your wheelchair.I can just picture it  https://t.co/uvl5HhbmbR
Tweet preprocesado: incredible beautifulbut laughed much read drifting wheelchairi picture 



### 3. Extracción de características

A continuación probarémos diferentes metodos de recodificar/transformar el input para futuros modelos. Es decir, el texto procesado de longitud variable se convertirá en vectores de características numéricas de tamaño fijo. Lo harémos mediante cuatro estrategias para transformar datos textuales en una representación numérica: Bolsa de palabras (Bag-of-Words), N-gramas de palabras. N-gramas de caracteres, Codificación TF-IDF y Word embeddings.

In [ ]:
# Formamos el vector x por 3 tweets aleatorios
x1 = preprocessing(tweets[2]) 
x2 = preprocessing(tweets[3])
x3 = preprocessing(tweets[4])
x = [x1,x2,x3]
print(x)

['wow skirt short length inch', 'incredible beautifulbut laughed much read drifting wheelchairi picture', 'find extremely hard believe kelly yr old mum would believe makes question fuck game especially none apologised tried explain amp immediate response like tweets attacking victims']


 #### 3.1 Bolsa de palabras (unigramas) ----------------------------------

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(x) #lista de textos
X_bag_of_words = vectorizer.transform(x) # Formamos la bolsa con los 3 tweets
X_bag_of_words #salida--> (nº frase, orden alfabético) : nº apariciones
print(vectorizer.vocabulary_) #diccionario con todas las palabras

{'wow': 36, 'skirt': 30, 'short': 29, 'length': 18, 'inch': 14, 'incredible': 15, 'beautifulbut': 3, 'laughed': 17, 'much': 21, 'read': 27, 'drifting': 5, 'wheelchairi': 34, 'picture': 25, 'find': 9, 'extremely': 8, 'hard': 12, 'believe': 4, 'kelly': 16, 'yr': 37, 'old': 24, 'mum': 22, 'would': 35, 'makes': 20, 'question': 26, 'fuck': 10, 'game': 11, 'especially': 6, 'none': 23, 'apologised': 1, 'tried': 31, 'explain': 7, 'amp': 0, 'immediate': 13, 'response': 28, 'like': 19, 'tweets': 32, 'attacking': 2, 'victims': 33}


La representación numérica queda de la siguiente forma, el inconveniente es que el orden de las palabras en el texto se pierde.

In [ ]:
print(X_bag_of_words[0:2])

  (0, 14)	1
  (0, 18)	1
  (0, 29)	1
  (0, 30)	1
  (0, 36)	1
  (1, 3)	1
  (1, 5)	1
  (1, 15)	1
  (1, 17)	1
  (1, 21)	1
  (1, 25)	1
  (1, 27)	1
  (1, 34)	1


#### 3.2 N-gramas de palabras. N-gramas de caracteres -----------------------------------

Es una alternativa a unigramas para considerar el orden de las palabras. Además, es resistente a los errores ortográficos, que en el caso de los tweets es muy acertado.

In [ ]:
#Prueba con letras 

vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))
vectorizer.fit([x1]) #utilizamos un solo texto
X_bag_of_words = vectorizer.transform([x1])
X_bag_of_words #salida: (nº frase, orden alfabético)  nº apariciones
print(vectorizer.vocabulary_)

{' w': 3, 'wo': 23, 'ow': 16, 'w ': 22, ' s': 2, 'sk': 19, 'ki': 11, 'ir': 10, 'rt': 17, 't ': 20, 'sh': 18, 'ho': 8, 'or': 15, ' l': 1, 'le': 12, 'en': 5, 'ng': 14, 'gt': 6, 'th': 21, 'h ': 7, ' i': 0, 'in': 9, 'nc': 13, 'ch': 4}


In [ ]:
#Prueba 2-n

vectorizer = CountVectorizer(ngram_range=(2, 2))
vectorizer.fit(x) #lista de textos
X_bag_of_words = vectorizer.transform(x)
X_bag_of_words #salida: (nº frase, orden alfabético)  nº apariciones
print(vectorizer.vocabulary_) 

{'wow skirt': 34, 'skirt short': 29, 'short length': 28, 'length inch': 18, 'incredible beautifulbut': 15, 'beautifulbut laughed': 3, 'laughed much': 17, 'much read': 21, 'read drifting': 26, 'drifting wheelchairi': 6, 'wheelchairi picture': 32, 'find extremely': 10, 'extremely hard': 9, 'hard believe': 13, 'believe kelly': 4, 'kelly yr': 16, 'yr old': 35, 'old mum': 24, 'mum would': 22, 'would believe': 33, 'believe makes': 5, 'makes question': 20, 'question fuck': 25, 'fuck game': 11, 'game especially': 12, 'especially none': 7, 'none apologised': 23, 'apologised tried': 1, 'tried explain': 30, 'explain amp': 8, 'amp immediate': 0, 'immediate response': 14, 'response like': 27, 'like tweets': 19, 'tweets attacking': 31, 'attacking victims': 2}


In [ ]:
#Prueba 5-n

vectorizer = CountVectorizer(ngram_range=(5, 5))
vectorizer.fit(x) #lista de textos
X_bag_of_words = vectorizer.transform(x)
X_bag_of_words #salida: (nº frase, orden alfabético)  nº apariciones
print(vectorizer.vocabulary_) 

{'wow skirt short length inch': 25, 'incredible beautifulbut laughed much read': 13, 'beautifulbut laughed much read drifting': 2, 'laughed much read drifting wheelchairi': 15, 'much read drifting wheelchairi picture': 17, 'find extremely hard believe kelly': 8, 'extremely hard believe kelly yr': 7, 'hard believe kelly yr old': 11, 'believe kelly yr old mum': 3, 'kelly yr old mum would': 14, 'yr old mum would believe': 26, 'old mum would believe makes': 20, 'mum would believe makes question': 18, 'would believe makes question fuck': 24, 'believe makes question fuck game': 4, 'makes question fuck game especially': 16, 'question fuck game especially none': 21, 'fuck game especially none apologised': 9, 'game especially none apologised tried': 10, 'especially none apologised tried explain': 5, 'none apologised tried explain amp': 19, 'apologised tried explain amp immediate': 1, 'tried explain amp immediate response': 23, 'explain amp immediate response like': 6, 'amp immediate response li

#### 3.3 Codificación TF-IDF ---------------------------------

Expresa cuán relevante es una palabra para un texto en un
corpus. De esta forma, el valor de un término aumenta proporcionalmente al número de veces
que aparece en el texto, y es compensado por su frecuencia en el corpus.

In [ ]:
#TfidfTransformer: Transforma una matriz de conteo en una representación TF-IDF.

pipe = Pipeline([('count', CountVectorizer()),('tfid', TfidfTransformer())])
pipe.fit(x)

Pipeline(memory=None,
         steps=[('count',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfid',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True))],
         verbose=False)

In [ ]:
#TfidfVectorizer: Convierte una colección de textos sin procesar en una matriz TF-IDF

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit([tweets[0],tweets[1]])
tfidf_vectorizer.transform([tweets[0],tweets[1]]).toarray()

array([[0.17674221, 0.17674221, 0.        , 0.17674221, 0.        ,
        0.        , 0.        , 0.17674221, 0.        , 0.        ,
        0.        , 0.        , 0.17674221, 0.17674221, 0.        ,
        0.17674221, 0.17674221, 0.        , 0.17674221, 0.17674221,
        0.        , 0.        , 0.17674221, 0.        , 0.        ,
        0.        , 0.17674221, 0.        , 0.        , 0.17674221,
        0.        , 0.17674221, 0.        , 0.17674221, 0.        ,
        0.17674221, 0.12575354, 0.17674221, 0.        , 0.        ,
        0.        , 0.        , 0.17674221, 0.        , 0.35348442,
        0.        , 0.17674221, 0.17674221, 0.        , 0.17674221,
        0.17674221, 0.        , 0.        , 0.12575354, 0.        ,
        0.        , 0.        , 0.        , 0.17674221, 0.17674221,
        0.        , 0.17674221, 0.        , 0.        , 0.17674221,
        0.17674221],
       [0.        , 0.        , 0.38244897, 0.        , 0.12748299,
        0.12748299, 0.12748

#### 3.4 Word embeddings ---------------------------------

Es un modelo más reciente que crea vectores para cada una de las palabras usando redes neuronales. El resultado es un conjunto de vectores de palabras donde los vectores cercanos en el espacio vectorial tienen significados similares basados ​​en el contexto, y los vectores de palabras distantes entre sí tienen significados diferentes.

In [ ]:
import gensim
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

def word2Vec(word):
    model = api.load("glove-twitter-25")
    vec = model.get_vector(word)
    return vec

In [ ]:
print(word2Vec(x1[0])) # para la palabra "incredible"

[-0.13495   0.90243   0.39679  -0.53241  -0.47991  -0.31209  -0.10571
  1.0359    0.91999   0.90929  -0.93797   0.41424  -4.2842   -0.080945
 -1.0815   -1.0538    0.32917   0.39186  -0.34039   0.86967  -0.32634
 -0.80405   0.031681  0.70784   0.2564  ]


Ahora entrenamos un dos tipos de modelo Word2Vec con el corpus formado por los 3 tweets aleatorios que cogimos al principio, para no sobrecargar el notebook. La parte principal del modelo es model.wv, donde “wv” significa “word vectors”.

> Modelo 1: palabras

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()
print('Nº de cores: ', cores)

Nº de cores:  2


In [ ]:
#Creamos una lista de este estilo para poder pasársela al modelo
x2 = [word_tokenize(i) for i in x]
print(x2)

[['wow', 'skirt', 'short', 'length', 'inch'], ['incredible', 'beautifulbut', 'laughed', 'much', 'read', 'drifting', 'wheelchairi', 'picture'], ['find', 'extremely', 'hard', 'believe', 'kelly', 'yr', 'old', 'mum', 'would', 'believe', 'makes', 'question', 'fuck', 'game', 'especially', 'none', 'apologised', 'tried', 'explain', 'amp', 'immediate', 'response', 'like', 'tweets', 'attacking', 'victims']]


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=x2, window=5, min_count=1, workers=cores-1)

In [ ]:
model.train(x, total_examples=model.corpus_count, epochs=30, report_delay=1) #pasamos la lista de tweets
model.wv.similarity('believe', 'game')

0.10731904

In [ ]:
model.wv.most_similar(positive=["mum"])

[('picture', 0.2748515009880066),
 ('fuck', 0.1919270157814026),
 ('hard', 0.14750374853610992),
 ('attacking', 0.13505050539970398),
 ('yr', 0.12198891490697861),
 ('especially', 0.10104119777679443),
 ('extremely', 0.0963210016489029),
 ('question', 0.09349123388528824),
 ('short', 0.09020862728357315),
 ('wow', 0.0766962543129921)]

> Modelo 2: bigramas

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
# Train a bigram detector.
bigram_transformer = Phrases(x2)

# Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
model2 = Word2Vec(bigram_transformer[x2], min_count=1)
model2.train(x, total_examples=model.corpus_count, epochs=30, report_delay=1) #pasamos la lista de tweets

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


(0, 8220)

In [ ]:
bigram_phraser = Phraser(bigram_transformer)
print(bigram_phraser)

Como podemos observar, el modelo no identifica ningún bigrama importante en el conjunto de tweets seleccionados.

In [ ]:
model2.wv.similarity('believe', 'game')

0.10731904

Los resultados varían ligeramente de un modelo a otro.

### 4. Construcción de clasificador SVM.

Antes de meternos en construcción de modelos hay que comprobar si se trata de una tarea de clasificación de clases balanceadas o no, en este caso consideramos que la clase 'sexista' y 'no-sexista' están balanceadas.

In [ ]:
# Vemos que las clases son  balanceadas 
from collections import Counter
Counter(t1)

Counter({'non-sexist': 1800, 'sexist': 1636})

Aplicamos el preproceso anterior:

In [ ]:
clean_tweets=[preprocessing(i) for i in tweets]
print(len(clean_tweets)) 

3436


Para recodificar el input de nuestro futuro clasificador hemos optado por la codificación TF-IDF con TfidfVectorizer.

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
x = tfidf_vectorizer.fit_transform(clean_tweets)
y = [ 1 if i=='sexist' else 0 for i in t1 ]

Vamos a entrenar un modelo SVM probando difentes kernels y valores de c para ver cuál es el más óptimo de todos. Esta es la función que comprueba las diferentes opciones. Para no hacer un triple bucle hemos decidido suponer que en el caso de kernel=poly cogerémos el degree por defecto, es decir 3, ya que iterando diferentes grados para ese kernel tarda demasiado en ejecutarse.

In [ ]:
def entrenar_SVM(X,Y, kernel, cv, c):  
    clf = svm.SVC(kernel=kernel, C=c,
                    gamma='scale')  
    args = {'kernel': kernel}
    clf.fit(X, Y)
    scores = cross_val_score(clf, X, Y, cv=cv)
    score_medio = scores.mean()
    errores=[]
    for score in scores:
        errores.append(1-score)
    mse = np.array(errores).mean()
    

    aux = 1.0 * np.array(errores)
    media_errores = aux.mean()
    # intervalo de confianza
    #epsilon = 1.96 * math.sqrt((media_errores * 
                  #(1 - media_errores)) / len(X)) 

    return clf, score_medio, mse, args

A continuación vemos que el modelo que mejor accuracy da es el SVM con kernel lineal, C=1 y validación cruzada con con fold=5.También habíamos probado con el perceptrón multicapa y regresión logística pero ambos modelos daban un accuracy mucho menor y la red neuronal además tardaba muchísimo en entrenarse.     


No se ha añadido ningún parámetro de class weight en el modelo ni se ha hecho ningún preproceso de over o undersampling ya que las clases están balanceadas.

In [ ]:
kernels = ['linear', 'rbf', 'poly', 'sigmoid']
C=[0.01, 0.5, 1, 10]
CV=5 
current_score = 0
for kernel in kernels:
    for c in C:
        clf, score_medio, mse, args = entrenar_SVM(x, y, kernel, CV, c)
        
        if score_medio > current_score:
            current_score = score_medio
            clf_mejor = clf
            score_medio_mejor = score_medio
            mse_mejor = mse
            args_best = args
            
print(f'Mejor modelo {clf_mejor} con {args_best}')
print(f'El error obtenido del mejor modelo es: {1-score_medio_mejor}')

print(
    f'Mejor resultado (accuracy): {score_medio_mejor} '
)


Mejor modelo SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) con {'kernel': 'linear'}
El error obtenido del mejor modelo es: 0.2753232795098338
Mejor resultado (accuracy): 0.7246767204901662 


Ahora aplicamos la misma función para encontrar un modelo de SVM que mejor prediga los niveles de sexismo, para eso recodificamos a numérica la variable task2 y aplicamos la misma función:


In [ ]:
pd.value_counts(t2)

non-sexist                      1800
ideological-inequality           386
stereotyping-dominance           366
sexual-violence                  344
misogyny-non-sexual-violence     284
objectification                  256
dtype: int64

In [ ]:
y2=[]
for i in t2:
  if i == 'non-sexist':
    y2.append(0)
  elif i == 'ideological-inequality':
    y2.append(1)
  elif i == 'stereotyping-dominance':
    y2.append(2)
  elif i ==  'sexual-violence' :
    y2.append(3)
  elif i == 'misogyny-non-sexual-violence':
    y2.append(4)
  else:
    y2.append(5)

In [ ]:
kernels = ['linear', 'rbf', 'poly', 'sigmoid']
C=[0.01, 0.5, 1, 10]
CV=5 
current_score = 0
for kernel in kernels:
    for c in C:
        clf, score_medio, mse, args = entrenar_SVM(x, y2, kernel, CV, c)
        
        if score_medio > current_score:
            current_score = score_medio
            clf_mejor = clf
            score_medio_mejor = score_medio
            mse_mejor = mse
            args_best = args
            
print(f'Mejor modelo {clf_mejor} con {args_best}')
print(f'El error obtenido del mejor modelo es: {1-score_medio_mejor}')

print(
    f'Mejor resultado (accuracy): {score_medio_mejor} '
)


Mejor modelo SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) con {'kernel': 'linear'}
El error obtenido del mejor modelo es: 0.39580413662367564
Mejor resultado (accuracy): 0.6041958633763244 


Vemos que para predecir los niveles de sexismo el accuracy es bastante peor. La mejor opción sigue siendo kernel = 'linear' y C=1

A continuación vamos a probar GridSearch también por si a caso para ver si con otra combinación de parámetros conseguimos mejor accuracy medio.

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(estimator=svm.SVC(), cv=5, param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})
clf.fit(x, y)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
clf.cv_results_

{'mean_fit_time': array([1.11858964, 1.34735632, 1.29625525, 1.35825481]),
 'mean_score_time': array([0.23969054, 0.28445454, 0.22347679, 0.29165301]),
 'mean_test_score': array([0.72467672, 0.71071011, 0.68684032, 0.71739955]),
 'param_C': masked_array(data=[1, 1, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'}],
 'rank_test_score': array([1, 3, 4, 2], dtype=int32),
 'split0_test_score': array([0.73546512, 0.71075581, 0.70348837, 0.73255814]),
 'split1_test_score': array([0.72925764, 0.72634643, 0.69432314, 0.72343523]),
 'split2_test_score': array([0.72197962, 0.70596798, 0.71179039, 0.71906841]),
 'split3_test_score': arra

Vemos que la mejor combinación de parámetros que devuelve GridSearch es la misma con la que dimos nosotras mediante nuestra función y efectivamente da el mismo accuracy que teníamos hasta ahora:

In [ ]:
clf.best_estimator_

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
clf.best_score_

0.7246767204901662

### 5. BERT

A partir de ahora vamos a probar el modelo de Bert para ver si mejoramos el accuracy, para eso volvemos a importar los datos sin preprocesarlos. Será necesario convertir la variable target en una variable numérica. Además al ser posible utilizar el modelo Bert_multilingual, en este apartado utilizaremos tanto los tweets ingleses como los tweets españoles para entrenar el modelo.

**Utilizaremos la librería Transformers de hugging face para nuestro objetivo:**

In [ ]:
!pip install transformers
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from transformers import pipeline #The easiest way to use a pretrained model on a given task is to use pipeline().
!pip install pytorch_lightning
import pytorch_lightning as pl 
from pytorch_lightning import Trainer
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from textwrap import wrap

     |████████████████████████████████| 2.3MB 30.8MB/s 
     |████████████████████████████████| 3.3MB 47.6MB/s 
     |████████████████████████████████| 901kB 49.3MB/s 
     |████████████████████████████████| 808kB 36.1MB/s 
     |████████████████████████████████| 829kB 43.2MB/s 
     |████████████████████████████████| 276kB 41.6MB/s 
     |████████████████████████████████| 112kB 38.2MB/s 
     |████████████████████████████████| 645kB 42.7MB/s 
     |████████████████████████████████| 1.3MB 39.2MB/s 
     |████████████████████████████████| 296kB 51.6MB/s 
     |████████████████████████████████| 143kB 50.0MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=2a4df517b0005031189d1f1e7f197e819df866c46281710717355af888f7c51f
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully un

In [ ]:
df = pd.read_csv('EXIST2021_training.tsv', sep='\t')
dftest = pd.read_csv('EXIST2021_test.tsv', sep='\t')
#df = df[df['language'] == 'en'] 
df['task1'] = df['task1'].replace({'sexist': 1, 'non-sexist':  0})  # Codificamos la variable target a numérica
df['task2'] = df['task2'].replace({'non-sexist': 0, 'ideological-inequality':  1, 'stereotyping-dominance': 2, 'sexual-violence': 3, 'misogyny-non-sexual-violence': 4, 'objectification': 5 }) 


In [1]:
 #Se importa el conjunto test para pasarlo posteriormente al modelo entrenado y guardar las predicciones las que se enviarán al concurso


#df = pd.read_csv('EXIST2021_test.tsv', sep='\t')
#df = df[df['language'] == 'en'] 
#df['task1'] = df['task1'].replace({'sexist': 1, 'non-sexist':  0})  # Codificamos la variable target a numérica

Implementación del modelo preentrenado BERT de Transformers con PyTorch con ayuda del tutorial (https://www.youtube.com/watch?v=mvh7DV84mr4):

In [ ]:
# Inicialización parámetros.


RANDOM_SEED = 58
MAX_LEN = 160 # basándonos en la longitud de los tokens tras el proceso de tokenización
BATCH_SIZE = 16
NCLASSES = 6 # 2 clases en la variable respuesta task1, 6 clases en la variable task2

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



print(device)


cuda:0


In [ ]:
# Importamos el modelo de interes y su tokenizador. Hemos elegido la versión de Bert que distingue entre mayúsculas y minúsculas.
PRE_TRAINED_MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
# CREACIÓN DATASET

class IMDBDataset(Dataset):

  def __init__(self,reviews,labels,tokenizer,max_len):
    self.reviews = reviews
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
      return len(self.reviews)
    
  def __getitem__(self, item):
    review = str(self.reviews[item])
    label = self.labels[item]
    encoding = tokenizer.encode_plus(
        review,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )

    return {
          'review': review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'label': torch.tensor(label, dtype=torch.long)
      }

In [ ]:
# Data loader:

def data_loader(df, tokenizer, max_len, batch_size):
  dataset = IMDBDataset(
      reviews = df.text.to_numpy(),
      labels = df.task2.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )

  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 4)

In [ ]:
# Realizamos la partición train - validación.
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# EL MODELO!

class BERTSentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BERTSentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.1)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, cls_output = self.bert(
        return_dict=False,
        input_ids = input_ids,
        attention_mask = attention_mask
    )
    drop_output = self.drop(cls_output)
    output = self.linear(drop_output)
    return output


In [ ]:
model = BERTSentimentClassifier(NCLASSES)
model = model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# ENTRENAMIENTO
# (For a typical Pytorch training cycle, we need to implement the loop for epochs, iterate through the mini-batches, 
# perform feedforward pass for each mini-batch, compute the loss, 
# perform backpropagation for each batch and then finally update the gradients.)

EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
# Iteración entrenamiento
def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double()/n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids = input_ids, attention_mask = attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)

In [ ]:
# Entrenamiento!!!

for epoch in range(EPOCHS):
  print('Epoch {} de {}'.format(epoch+1, EPOCHS))
  print('------------------')
  train_acc, train_loss = train_model(
      model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
  )
  test_acc, test_loss = eval_model(
      model, test_data_loader, loss_fn, device, len(df_test)
  )
  print('Entrenamiento: Loss: {}, accuracy: {}'.format(train_loss, train_acc))
  print('Validación: Loss: {}, accuracy: {}'.format(test_loss, test_acc))

Epoch 1 de 5
------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/py

Entrenamiento: Loss: 1.299458673570764, accuracy: 0.5547392940333273
Validación: Loss: 1.15576252409003, accuracy: 0.5616045845272206
Epoch 2 de 5
------------------


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Entrenamiento: Loss: 0.9559501386474402, accuracy: 0.6620677297975274
Validación: Loss: 1.144628637216308, accuracy: 0.5773638968481375
Epoch 3 de 5
------------------


...

Entrenamiento: Loss: 0.7493488409030745, accuracy: 0.7412650062712776
Validación: Loss: 1.1533515107902614, accuracy: 0.6060171919770774
Epoch 4 de 5
------------------


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Entrenamiento: Loss: 0.6038289842513366, accuracy: 0.7994982977960939
Validación: Loss: 1.1794308677993037, accuracy: 0.6181948424068767
Epoch 5 de 5
------------------


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

Entrenamiento: Loss: 0.5169852087781217, accuracy: 0.8342590933524457
Validación: Loss: 1.1962187757546252, accuracy: 0.6131805157593123


In [ ]:
# Para guardar el modelo previamente entrenado

#torch.save(model.state_dict(), 'bertMultilingual.bin')
#modelo = torch.load('bertMultilingual.bin')

In [ ]:
def classifySentiment(review_text):
  encoding_review = tokenizer.encode_plus(
      review_text,
      max_length = MAX_LEN,
      truncation = True,
      add_special_tokens = True,
      return_token_type_ids = False,
      pad_to_max_length = True,
      return_attention_mask = True,
      return_tensors = 'pt'
      )
  
  input_ids = encoding_review['input_ids'].to(device)
  attention_mask = encoding_review['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)
  #print("\n".join(wrap(review_text)))
  return (prediction)


Predecimos y guardamos las predicciones en el formato solicitado por la competición:

In [ ]:
dftest['predicciones'] = dftest['text'].map(classifySentiment)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dftest.tail()

,test_case,id,source,language,text,predicciones
4363,EXIST2021,11341,twitter,es,"@IreneMontero Se llama nota de corte, y es lo ...","[tensor(2, device='cuda:0')]"
4364,EXIST2021,11342,twitter,es,@freckles887 Osea todo atack of titan parte de...,"[tensor(0, device='cuda:0')]"
4365,EXIST2021,11343,twitter,es,@alabadomango Cuéntame más!!Es por androcentri...,"[tensor(0, device='cuda:0')]"
4366,EXIST2021,11344,gab,es,Que duro es ser tan atractiva como Jaba de Hutt.,"[tensor(0, device='cuda:0')]"
4367,EXIST2021,11345,twitter,es,@elmundoes A Pablo es que ya no le hacen caso ...,"[tensor(0, device='cuda:0')]"


In [ ]:
def recodificar(x):
  if str(x)[8] == '0':
    return 'non-sexist'
  elif str(x)[8] == '1':
    return 'ideological-inequality'
  elif str(x)[8] == '2':
    return 'stereotyping-dominance'
  elif str(x)[8] == '3':
    return 'sexual-violence'
  elif str(x)[8] == '4':
    return 'misogyny-non-sexual-violence'
  else:
    return 'objectification'

def id(x):
  return str(x).zfill(6)

#df['task2'] = df['task2'].replace({'non-sexist': 0, 'ideological-inequality':  1, 'stereotyping-dominance': 2, 'sexual-violence': 3, 'misogyny-non-sexual-violence': 4, 'objectification': 5 }) 


dftest['predicciones'] = dftest['predicciones'].apply(recodificar)
dftest['id'] = dftest['id'].apply(id)

In [ ]:
#dftest.head(100)
dftest = dftest.drop(['source','language', 'text'], axis=1)
dftest.to_csv('task2_Andrea_Lisa_1.tsv',sep='\t', header=None, index=False)

In [ ]:
def id(x):
  return str(x).zfill(6)
task1 = pd.read_csv('task1_Andrea_Lisa_1.tsv', sep='\t', header=None)
task1[1] = task1[1].apply(id)


In [ ]:
task1[1] = task1[1].astype(str)
task1[1]

0       006978
1       006979
2       006980
3       006981
4       006982
         ...  
4363    011341
4364    011342
4365    011343
4366    011344
4367    011345
Name: 1, Length: 4368, dtype: object

In [ ]:
#task1 = pd.read_csv('task1_Andrea_Lisa_1.tsv', sep='\t', header=None)
#task1[2] = task1[2].replace({'sexist': 'non-sexist', 'non-sexist':  'sexist'})
#task1.head()
task1.to_csv('task1_Andrea_Lisa_1.tsv',sep='\t', header=None, index=False)

In [ ]:
dftest.to_csv('task1_Andrea_Lisa_1.tsv',sep='\t', header=None, index=False)

Vemos que por ahora es el mejor accuracy obtenido.



### 7. StackingClassifier

La idea de Stacking implica el entrenamiento de un modelo para combinar las predicciones de otros modelos. 

Se comienza entrenando varios aprendices sobre los datos de entrenamient (modelos de primer nivel), y se termina entrenando el modelo final (modelo de segundo nivel o stacking model) sobre los datos originales considerando como características adicionales las predicciones de los primeros. Este modelo final es frecuentemente una regresión logística.  

El rendimiento de este método de ensamblado aumenta cuanto más diversos sean los modelos de primer nivel, ya que cada uno de estos modelos explicará una parte diferente de la varianza de los datos originales.

In [ ]:
from sklearn.svm import SVC
from sklearn import model_selection

svm = SVC() 
lr = LogisticRegression()
cart = DecisionTreeClassifier() 
rf = RandomForestClassifier()
mlp = MLPClassifier(random_state=1, max_iter=300)
knn = KNeighborsClassifier()

estimators = [('svm', svm),('lr', lr),('cart', cart),('rf', rf),('mlp',mlp),('knn',knn)]

sclf = StackingClassifier(estimators=estimators, final_estimator=lr)



Comparamos los modelos para Task1:

In [ ]:
for clf, label in zip([svm, lr, cart, rf, mlp, knn, sclf], ['SVC', 'Logistic Regression','Decision Tree','Random Forest', 'MLPClassifier','KNN', 'StackingClassifier']):
    scores = model_selection.cross_val_score(clf, x, y, cv=3, scoring='accuracy')
    print("Accuracy: %f [%s]"  % (scores.mean(), label))

Accuracy: 0.699359 [SVC]
Accuracy: 0.699651 [Logistic Regression]
Accuracy: 0.679574 [Decision Tree]
Accuracy: 0.712166 [Random Forest]
Accuracy: 0.648137 [MLPClassifier]
Accuracy: 0.628634 [KNN]
Accuracy: 0.726716 [StackingClassifier]


Comparamos los modelos para Task2:

In [ ]:
for clf, label in zip([svm, lr, cart, rf, mlp, knn, sclf], ['SVC', 'Logistic Regression','Decision Tree','Random Forest', 'MLPClassifier','KNN', 'StackingClassifier']):
    scores = model_selection.cross_val_score(clf, x, y2, cv=3, scoring='accuracy')
    print("Accuracy: %f [%s]"  % (scores.mean(), label))

Accuracy: 0.561990 [SVC]
Accuracy: 0.569850 [Logistic Regression]
Accuracy: 0.550641 [Decision Tree]
Accuracy: 0.601572 [Random Forest]
Accuracy: 0.557625 [MLPClassifier]
Accuracy: 0.555586 [KNN]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.621074 [StackingClassifier]


Vamos a evaluar las predicciones de task2 con la métrica f1-macro ya que es más robusta que el accuracy en este tipo de clasificaciones:

In [ ]:
for clf, label in zip([svm, lr, cart, rf, mlp, knn, sclf], ['SVC', 'Logistic Regression','Decision Tree','Random Forest', 'MLPClassifier','KNN', 'StackingClassifier']):
    scores = model_selection.cross_val_score(clf, x, y2, cv=3, scoring='f1_macro')
    print("F1 score: %f [%s]"  % (scores.mean(), label))

F1 score: 0.252657 [SVC]
F1 score: 0.286128 [Logistic Regression]
F1 score: 0.416182 [Decision Tree]
F1 score: 0.386311 [Random Forest]
F1 score: 0.375267 [MLPClassifier]
F1 score: 0.391629 [KNN]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


F1 score: 0.450914 [StackingClassifier]
